#  Data Process

##   Extract: Date, New_cases, New_Deaths, Recovered(without US)

###    Update & Download files

In [1]:
import os 
import pandas as pd


# %%time
import requests

for filename in ['time_series_covid19_confirmed_global.csv',
                 'time_series_covid19_deaths_global.csv',
                 'time_series_covid19_recovered_global.csv',
                 'time_series_covid19_confirmed_US.csv',
                 'time_series_covid19_deaths_US.csv']:
    if os.path.exists(filename):
        os.remove(filename)
    print(f'Downloading {filename}')
    url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/{filename}'
    myfile = requests.get(url)
    open(filename, 'wb').write(myfile.content)

###  Setup: 在这里输入想要的国家名，就可以在result文件夹中得到结果（美国除外）

In [2]:
countries = ['China', 'Japan','Brazil','India']

###   Read and Convert files

In [3]:
from datetime import datetime

def convert_date_str(df):
    try:
        df.columns = list(df.columns[:4]) + [datetime.strptime(d, "%m/%d/%y").date().strftime("%Y-%m-%d") for d in df.columns[4:]]
    except:
        print('_convert_date_str failed with %y, try %Y')
        df.columns = list(df.columns[:4]) + [datetime.strptime(d, "%m/%d/%Y").date().strftime("%Y-%m-%d") for d in df.columns[4:]]

        
confirmed_global_df = pd.read_csv('time_series_covid19_confirmed_global.csv')
convert_date_str(confirmed_global_df)

deaths_global_df = pd.read_csv('time_series_covid19_deaths_global.csv')
convert_date_str(deaths_global_df)

recovered_global_df = pd.read_csv('time_series_covid19_recovered_global.csv')
convert_date_str(recovered_global_df)

confirmed_global_df

,Province/State,Country/Region,Lat,Long,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2021-08-20,2021-08-21,2021-08-22,2021-08-23,2021-08-24,2021-08-25,2021-08-26,2021-08-27,2021-08-28,2021-08-29
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,152448,152448,152448,152583,152660,152722,152822,152960,152960,152960
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,138132,138790,139324,139721,140521,141365,142253,143174,144079,144847
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,190656,191171,191583,192089,192626,193171,193674,194186,194671,195162
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,14988,14988,14988,15002,15003,15014,15016,15025,15025,15025
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,45583,45817,45945,46076,46340,46539,46726,46929,47079,47168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,NaN,Vietnam,14.058324,108.277199,0,2,2,2,2,2,...,323268,336707,348059,358456,369267,381363,392938,410366,422469,435265
275,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,324542,325404,326310,327634,329204,330821,332609,334472,335709,337405
276,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,7454,7487,7509,7539,7580,7625,7676,7711,7751,7784
277,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,203978,204337,204549,204651,204977,205107,205315,205704,205893,206051


###  Filter out problematic data points

In [5]:
import numpy as np


removed_states = "Recovered|Grand Princess|Diamond Princess"
removed_countries = "US|The West Bank and Gaza"

confirmed_global_df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True)
deaths_global_df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True)
recovered_global_df.rename(columns={"Province/State": "Province_State", "Country/Region": "Country_Region"}, inplace=True)

# confirmed_global_df = confirmed_global_df[~confirmed_global_df["Province_State"].replace(np.nan, "nan").str.match(removed_states)]
# deaths_global_df    = deaths_global_df[~deaths_global_df["Province_State"].replace(np.nan, "nan").str.match(removed_states)]
# recovered_global_df = recovered_global_df[~recovered_global_df["Province_State"].replace(np.nan, "nan").str.match(removed_states)]

# confirmed_global_df = confirmed_global_df[~confirmed_global_df["Country_Region"].replace(np.nan, "nan").str.match(removed_countries)]
# deaths_global_df    = deaths_global_df[~deaths_global_df["Country_Region"].replace(np.nan, "nan").str.match(removed_countries)]
# recovered_global_df = recovered_global_df[~recovered_global_df["Country_Region"].replace(np.nan, "nan").str.match(removed_countries)]


### Data Preprocess(melt data)

In [6]:
confirmed_global_melt_df = confirmed_global_df.melt(
    id_vars=['Country_Region', 'Province_State'], value_vars=confirmed_global_df.columns[4:], var_name='Date', value_name='ConfirmedCases')
deaths_global_melt_df = deaths_global_df.melt(
    id_vars=['Country_Region', 'Province_State'], value_vars=confirmed_global_df.columns[4:], var_name='Date', value_name='Deaths')
recovered_global_melt_df = recovered_global_df.melt(
    id_vars=['Country_Region', 'Province_State'], value_vars=confirmed_global_df.columns[4:], var_name='Date', value_name='Recovered')

recovered_global_melt_df

,Country_Region,Province_State,Date,Recovered
0,Afghanistan,NaN,2020-01-22,0
1,Albania,NaN,2020-01-22,0
2,Algeria,NaN,2020-01-22,0
3,Andorra,NaN,2020-01-22,0
4,Angola,NaN,2020-01-22,0
...,...,...,...,...
154699,Vietnam,NaN,2021-08-29,0
154700,West Bank and Gaza,NaN,2021-08-29,0
154701,Yemen,NaN,2021-08-29,0
154702,Zambia,NaN,2021-08-29,0


###  Data Preprocess(merge data)

In [7]:
merge_data = confirmed_global_melt_df.merge(deaths_global_melt_df, on=['Country_Region', 'Province_State', 'Date'])
merge_data = merge_data.merge(recovered_global_melt_df, on=['Country_Region', 'Province_State', 'Date'])

merge_data

,Country_Region,Province_State,Date,ConfirmedCases,Deaths,Recovered
0,Afghanistan,NaN,2020-01-22,0,0,0
1,Albania,NaN,2020-01-22,0,0,0
2,Algeria,NaN,2020-01-22,0,0,0
3,Andorra,NaN,2020-01-22,0,0,0
4,Angola,NaN,2020-01-22,0,0,0
...,...,...,...,...,...,...
154113,Vietnam,NaN,2021-08-29,435265,10749,0
154114,West Bank and Gaza,NaN,2021-08-29,337405,3672,0
154115,Yemen,NaN,2021-08-29,7784,1461,0
154116,Zambia,NaN,2021-08-29,206051,3596,0


### Save [Country, Date,ConfirmedCases,Deaths,Recovered] as files

In [8]:
df = merge_data.drop(['Province_State'], inplace=False, axis=1)
for country in countries:
    df_result = df[df['Country_Region']==country]
    df_result1 = df_result.groupby(['Country_Region','Date'])\
    [['ConfirmedCases', 'Deaths','Recovered']].sum().reset_index()
    
    # save file
    df_result1.to_csv('./result/data_'+country+'.csv')
    